In [14]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
# 将 src 目录添加到 Python 路径，以便导入自定义模块
# 如果此脚本不在项目根目录，请相应调整路径
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '.'))) # 如果直接运行脚本
# 如果在 Notebook 中，确保 Notebook 的工作目录是项目根目录，或者使用下面的方式
try:
    from src.data_processing.processor import DataProcessor
    from src.visualization.plotter import Plotter
except ImportError:
    # 尝试添加上一级目录到 sys.path (如果是在类似 'scripts' 的子文件夹中运行)
    # 或者直接确保你的 PYTHONPATH 设置正确
    current_dir = os.getcwd() # 如果在 notebook 中，这通常是 notebook 文件的位置
    project_root = os.path.dirname(current_dir) # 假设 notebook 在类似 'notebooks' 文件夹
    if 'src' not in sys.path: # 避免重复添加
         # 根据实际结构调整，确保能找到 src 目录
        # 如果项目结构是 root/src, root/notebooks，那么从 notebooks 目录出发：
        project_root_for_src = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) # 示例
        # 或者更简单的方式是，确保运行此代码时，工作目录是项目的根目录
        # 例如，如果你的项目根目录是 'disbusiness'，在 'disbusiness' 下启动 jupyter notebook
        # 或者手动指定 sys.path.append('/path/to/your/disbusiness')
        # 为了简单起见，这里假设 src 就在当前工作目录的子目录中，或者 PYTHONPATH 已配置
        try:
            # 这个路径假定您在项目根目录执行此代码块，或者 PYTHONPATH 已包含项目根目录
            sys.path.insert(0, os.path.abspath('../notebooks'))
            from src.data_processing.processor import DataProcessor
            from src.visualization.plotter import Plotter
        except ImportError as e:
            print(f"导入模块失败，请确保您的 Python 环境配置正确，并且 src 目录在 PYTHONPATH 中。错误: {e}")
            print(f"当前工作目录: {os.getcwd()}")
            print(f"Python 路径: {sys.path}")
            # 如果上面的尝试失败，并且你确定 plotter.py 和 processor.py 在 src 目录下，
            # 可以尝试硬编码一个相对路径（不推荐，但作为后备）
            # sys.path.append('./src') # 假设脚本在项目根目录运行
            # from visualization.plotter import Plotter
            # from data_processing.processor import DataProcessor
            # raise # 重新抛出错误，因为没有模块无法继续

def main():
    try:
        # 优先尝试 PingFang SC (苹果丽黑) 或 SimHei (黑体)
        plt.rcParams['font.sans-serif'] = ['PingFang SC', 'SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("已尝试应用 Matplotlib 中文字体设置 (PingFang SC, SimHei, Arial Unicode MS)。")
    except Exception as e:
            print(f"尝试设置中文字体时发生错误: {e}。请确保这些字体中至少有一个已安装。")
            print("将依赖 plotter.py 中的字体设置（如果它有不同的成功设置）。")
    print("--- 数据分析与可视化报告生成开始 ---")

    # 0. 定义路径
    RAW_DATA_PATH = '../data/raw/Online Retail.xlsx'
    PROCESSED_DATA_SAVE_PATH_PARQUET = 'data/processed/Online Retail_processed_from_script.parquet' # 新处理的数据保存路径
    FIGURES_SAVE_DIR = '../reports/figures'

    if not os.path.exists(FIGURES_SAVE_DIR):
        os.makedirs(FIGURES_SAVE_DIR)
        print(f"已创建目录: {FIGURES_SAVE_DIR}")

    # 1. 数据加载和预处理
    print("\\n--- 1. 数据加载和预处理 ---")
    data_processor = DataProcessor(data_path=RAW_DATA_PATH)

    raw_df = None
    try:
        raw_df = data_processor.load_data()
    except Exception as e:
        print(f"数据加载失败: {e}")
        return

    if raw_df is None or raw_df.empty:
        print("未能加载数据，程序终止。")
        return

    print("\\n原始数据基本信息:")
    raw_df.info(verbose=False) # 简略信息
    print(f"原始数据形状: {raw_df.shape}")

    # 清理数据，根据 processor.py 中的默认参数进行
    # 您可以根据需要调整 clean_data 的参数
    # 例如: handle_outliers_quantity=True, handle_outliers_unitprice=True
    # 为了演示，我们使用默认值，不进行异常值处理，以免过滤过多数据
    cleaned_df = data_processor.clean_data(
        raw_df,
        remove_cancelled_orders=True,
        remove_zero_unit_price=True,
        handle_outliers_quantity=False, # 改为 True 以处理数量异常值
        handle_outliers_unitprice=False # 改为 True 以处理单价异常值
    )

    if cleaned_df is None or cleaned_df.empty:
        print("数据清理后为空，程序终止。")
        return

    print("\\n清理后数据基本信息:")
    cleaned_df.info(verbose=False)
    print(f"清理后数据形状: {cleaned_df.shape}")

    # 数据转换和特征工程
    processed_df = data_processor.transform_data(cleaned_df)

    if processed_df is None or processed_df.empty:
        print("数据转换后为空，程序终止。")
        return

    print("\\n转换后数据基本信息 (包含特征工程):")
    processed_df.info(verbose=False)
    print(f"转换后数据形状: {processed_df.shape}")
    print("\\n转换后数据样本 (前5行):")
    print(processed_df.head().to_markdown(index=False))

    # 保存处理后的数据 (可选)
    try:
        data_processor.save_processed_data(output_path=PROCESSED_DATA_SAVE_PATH_PARQUET)
        # 如果 save_processed_data 内部没有打印，可以加一句
        print(f"处理后的数据已保存到: {PROCESSED_DATA_SAVE_PATH_PARQUET}")
    except Exception as e:
        print(f"保存处理后数据失败: {e}")


    # 2. 数据可视化
    print("\\n--- 2. 数据可视化 ---")
    plotter = Plotter(style='seaborn-v0_8-whitegrid') # 初始化 Plotter
    figure_paths = {}

    # a. 核心指标的分布
    print("\\n正在生成核心指标分布图...")
    if 'Quantity' in processed_df.columns:
        # 对 Quantity 进行筛选以便更好地可视化，例如排除极端值
        # 这里使用原始 Quantity，但如果分布很偏，考虑过滤或对数变换
        q_upper_bound = processed_df['Quantity'].quantile(0.99) # 排除最高的1%
        plotter.plot_histogram(processed_df[processed_df['Quantity'] <= q_upper_bound], 'Quantity',
                               title='商品销售数量分布 (排除前1%极端值)', xlabel='数量 (Quantity)',
                               save_path=os.path.join(FIGURES_SAVE_DIR, 'quantity_distribution.png'))
        figure_paths['quantity_distribution'] = os.path.join(FIGURES_SAVE_DIR, 'quantity_distribution.png')

    if 'UnitPrice' in processed_df.columns:
        up_upper_bound = processed_df['UnitPrice'].quantile(0.99) # 排除最高的1%
        plotter.plot_histogram(processed_df[processed_df['UnitPrice'] <= up_upper_bound], 'UnitPrice',
                               title='商品单价分布 (排除前1%极端值)', xlabel='单价 (UnitPrice)',
                               save_path=os.path.join(FIGURES_SAVE_DIR, 'unitprice_distribution.png'))
        figure_paths['unitprice_distribution'] = os.path.join(FIGURES_SAVE_DIR, 'unitprice_distribution.png')

    if 'TotalPrice' in processed_df.columns:
        tp_upper_bound = processed_df['TotalPrice'].quantile(0.99) # 排除最高的1%
        plotter.plot_histogram(processed_df[processed_df['TotalPrice'] <= tp_upper_bound], 'TotalPrice',
                               title='订单总价分布 (排除前1%极端值)', xlabel='总价 (TotalPrice)',
                               save_path=os.path.join(FIGURES_SAVE_DIR, 'totalprice_distribution.png'))
        figure_paths['totalprice_distribution'] = os.path.join(FIGURES_SAVE_DIR, 'totalprice_distribution.png')

    # b. 销售趋势 (按月)
    print("\\n正在生成销售趋势图...")
    if 'InvoiceDate' in processed_df.columns and 'TotalPrice' in processed_df.columns:
        # 按月聚合销售额
        # 确保 InvoiceDate 是 datetime 类型，transform_data 应该已经处理了
        monthly_sales = processed_df.groupby(processed_df['InvoiceDate'].dt.to_period('M'))['TotalPrice'].sum().reset_index()
        monthly_sales['InvoiceDate'] = monthly_sales['InvoiceDate'].dt.to_timestamp() # 转回 datetime 以便绘图

        plotter.plot_lineplot(monthly_sales, 'InvoiceDate', 'TotalPrice',
                              title='月度销售总额趋势', xlabel='月份', ylabel='销售总额',
                              save_path=os.path.join(FIGURES_SAVE_DIR, 'monthly_sales_trend.png'))
        figure_paths['monthly_sales_trend'] = os.path.join(FIGURES_SAVE_DIR, 'monthly_sales_trend.png')

    # c. 按国家/地区分析
    print("\\n正在生成按国家/地区分析图...")
    if 'Country' in processed_df.columns and 'TotalPrice' in processed_df.columns:
        top_n_countries = 10
        country_sales = processed_df.groupby('Country')['TotalPrice'].sum().nlargest(top_n_countries).reset_index()
        plotter.plot_bar_chart(country_sales, 'Country', 'TotalPrice',
                               title=f'销售额最高的 Top {top_n_countries} 国家/地区', xlabel='国家/地区', ylabel='总销售额',
                               horizontal=True, # 水平条形图可能更好看
                               save_path=os.path.join(FIGURES_SAVE_DIR, 'top_countries_sales.png'))
        figure_paths['top_countries_sales'] = os.path.join(FIGURES_SAVE_DIR, 'top_countries_sales.png')

    # d. 畅销商品分析
    print("\\n正在生成畅销商品分析图...")
    if 'Description' in processed_df.columns and 'Quantity' in processed_df.columns:
        top_n_products = 10
        # 按描述统计总销量
        product_sales_quantity = processed_df.groupby('Description')['Quantity'].sum().nlargest(top_n_products).reset_index()
        plotter.plot_bar_chart(product_sales_quantity, 'Description', 'Quantity',
                               title=f'销量最高的 Top {top_n_products} 商品', xlabel='商品描述', ylabel='总销量',
                               horizontal=True,
                               save_path=os.path.join(FIGURES_SAVE_DIR, 'top_products_by_quantity.png'))
        figure_paths['top_products_by_quantity'] = os.path.join(FIGURES_SAVE_DIR, 'top_products_by_quantity.png')

    # e. 相关系数矩阵
    print("\\n正在生成相关系数矩阵...")
    # plotter.plot_correlation_matrix 接受的是整个 DataFrame，它内部会选择数值列
    # 确保 processed_df 包含期望的数值列，如 Quantity, UnitPrice, TotalPrice, OrderYear, OrderMonth 等
    # 'OrderHour' 可能也很有趣
    numerical_cols_for_corr = ['Quantity', 'UnitPrice', 'TotalPrice', 'OrderYear', 'OrderMonth', 'OrderDay', 'OrderHour']
    # 确保这些列存在且为数值类型
    valid_numerical_cols = [col for col in numerical_cols_for_corr if col in processed_df.columns and pd.api.types.is_numeric_dtype(processed_df[col])]

    if len(valid_numerical_cols) >= 2 : # 需要至少两个数值列来计算相关性
        plotter.plot_correlation_matrix(processed_df[valid_numerical_cols],
                                        title='核心数值特征相关系数矩阵',
                                        save_path=os.path.join(FIGURES_SAVE_DIR, 'correlation_matrix.png'))
        figure_paths['correlation_matrix'] = os.path.join(FIGURES_SAVE_DIR, 'correlation_matrix.png')
    else:
        print(f"警告: 数值列不足 ({valid_numerical_cols})，无法生成相关系数矩阵。")


    # 3. 输出图表路径和总结
    print("\\n--- 3. 图表和总结 ---")
    print("所有图表已生成并保存到 'reports/figures/' 目录下。")
    print("\\n生成的图表文件路径:")
    for name, path in figure_paths.items():
        print(f"- {name}: {path}")

    print("\\n\\n--- 数据分析关键摘要 ---")
    if processed_df is not None and not processed_df.empty:
        print(f"1. 数据概览: 清理和处理后，数据集包含 {processed_df.shape[0]} 条记录和 {processed_df.shape[1]} 个特征。")

        if 'TotalPrice' in processed_df.columns:
            total_revenue = processed_df['TotalPrice'].sum()
            print(f"2. 总销售额: {total_revenue:,.2f}")

        if 'InvoiceNo' in processed_df.columns:
            unique_invoices = processed_df['InvoiceNo'].nunique()
            print(f"3. 独立订单数量: {unique_invoices}")

        if 'CustomerID' in processed_df.columns:
            # 排除 'UNKNOWN' 客户进行统计
            known_customers_df = processed_df[processed_df['CustomerID'] != 'UNKNOWN']
            unique_customers = known_customers_df['CustomerID'].nunique()
            print(f"4. 独立客户数量 (已识别): {unique_customers}")

        if 'StockCode' in processed_df.columns:
            unique_products = processed_df['StockCode'].nunique()
            print(f"5. 独立产品种类 (按StockCode): {unique_products}")

        if 'Country' in processed_df.columns:
            country_with_most_sales = processed_df.groupby('Country')['TotalPrice'].sum().nlargest(1)
            if not country_with_most_sales.empty:
                print(f"6. 销售额最高的国家: {country_with_most_sales.index[0]} (销售额: {country_with_most_sales.iloc[0]:,.2f})")

        # 时间范围
        if 'InvoiceDate' in processed_df.columns:
            min_date = processed_df['InvoiceDate'].min()
            max_date = processed_df['InvoiceDate'].max()
            if pd.notnull(min_date) and pd.notnull(max_date):
                print(f"7. 数据时间范围: 从 {min_date.strftime('%Y-%m-%d')} 到 {max_date.strftime('%Y-%m-%d')}")

        print("\\n请将以上摘要和生成的图表整合到您的Word报告中。")
    else:
        print("由于数据处理失败或数据为空，无法生成摘要。")

    print("\\n--- 报告生成脚本执行完毕 ---")

if __name__ == '__main__':
    # 确保工作目录是项目根目录
    # 如果从根目录运行 `python your_script_name.py`，这通常是正确的
    # 如果在 IDE 中运行，也请检查运行配置
    # print(f"当前执行脚本的工作目录: {os.getcwd()}") # 用于调试
    main()


已尝试应用 Matplotlib 中文字体设置 (PingFang SC, SimHei, Arial Unicode MS)。
--- 数据分析与可视化报告生成开始 ---
\n--- 1. 数据加载和预处理 ---
数据成功从 ../data/raw/Online Retail.xlsx 加载。形状: (541909, 8)
\n原始数据基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Columns: 8 entries, InvoiceNo to Country
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
原始数据形状: (541909, 8)
开始数据清理...
已将 'InvoiceDate' 转换为 datetime。转换后的空值数量: 0
丢弃无效 InvoiceDate 后的行数: 541909
已处理 CustomerID 中的缺失值。对 135080 条目使用 'UNKNOWN'。
丢弃空 Description 后的行数: 541909
丢弃空 StockCode 后的行数: 541909
Quantity/UnitPrice 特定清理前的初始行数: 541909
移除了 10624 行 Quantity <= 0 的记录。
移除了 1181 行 UnitPrice <= 0 的记录。
丢弃了 5226 行重复记录。
数据清理完成。最终形状: (524878, 8)
\n清理后数据基本信息:
<class 'pandas.core.frame.DataFrame'>
Index: 524878 entries, 0 to 541908
Columns: 8 entries, InvoiceNo to Country
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 36.0+ MB
清理后数据形状: (524878, 8)
开始数据转换...
已创建 'TotalPrice' 列。
已提取日期特征: 年, 月, 日

/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 39057 (\N{CJK UNIFIED IDEOGRAPH-9891}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 21830 (\N{CJK UNIFIED IDEOGRAPH-5546}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 21697 (\N{CJK UNIFIED IDEOGRAPH-54C1}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 38144 (\N{CJK UNIFIED IDEOGRAPH-9500}) mi

图表已保存到 ../reports/figures/quantity_distribution.png


/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 39057 (\N{CJK UNIFIED IDEOGRAPH-9891}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 21830 (\N{CJK UNIFIED IDEOGRAPH-5546}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 21697 (\N{CJK UNIFIED IDEOGRAPH-54C1}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:69: UserWarning: Glyph 21333 (\N{CJK UNIFIED IDEOGRAPH-5355}) mi

图表已保存到 ../reports/figures/unitprice_distribution.png


...

图表已保存到 ../reports/figures/totalprice_distribution.png
\n正在生成销售趋势图...
图表已保存到 ../reports/figures/monthly_sales_trend.png
\n正在生成按国家/地区分析图...


/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:296: UserWarning: Glyph 24635 (\N{CJK UNIFIED IDEOGRAPH-603B}) missing from current font.
  plt.tight_layout() # 调整布局以防止标签重叠
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:296: UserWarning: Glyph 38144 (\N{CJK UNIFIED IDEOGRAPH-9500}) missing from current font.
  plt.tight_layout() # 调整布局以防止标签重叠
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:296: UserWarning: Glyph 21806 (\N{CJK UNIFIED IDEOGRAPH-552E}) missing from current font.
  plt.tight_layout() # 调整布局以防止标签重叠
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:296: UserWarning: Glyph 39069 (\N{CJK UNIFIED IDEOGRAPH-989D}) missing from current font.
  plt.tight_layout() # 调整布局以防止标签重叠
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:296: UserWarning: Gl

图表已保存到 ../reports/figures/top_countries_sales.png
\n正在生成畅销商品分析图...
图表已保存到 ../reports/figures/top_products_by_quantity.png
\n正在生成相关系数矩阵...


/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:185: UserWarning: Glyph 26680 (\N{CJK UNIFIED IDEOGRAPH-6838}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:185: UserWarning: Glyph 24515 (\N{CJK UNIFIED IDEOGRAPH-5FC3}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:185: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:185: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from current font.
  fig.savefig(save_path)
/Users/linhengwei/Desktop/lesson/大三课程/大三下课程/python数据分析/disbusiness/src/visualization/plotter.py:185: UserWarning: Glyph 29305 (\N{CJK UNIFIED IDEOGRAPH-7279

图表已保存到 ../reports/figures/correlation_matrix.png
\n--- 3. 图表和总结 ---
所有图表已生成并保存到 'reports/figures/' 目录下。
\n生成的图表文件路径:
- quantity_distribution: ../reports/figures/quantity_distribution.png
- unitprice_distribution: ../reports/figures/unitprice_distribution.png
- totalprice_distribution: ../reports/figures/totalprice_distribution.png
- monthly_sales_trend: ../reports/figures/monthly_sales_trend.png
- top_countries_sales: ../reports/figures/top_countries_sales.png
- top_products_by_quantity: ../reports/figures/top_products_by_quantity.png
- correlation_matrix: ../reports/figures/correlation_matrix.png
\n\n--- 数据分析关键摘要 ---
1. 数据概览: 清理和处理后，数据集包含 524878 条记录和 17 个特征。
2. 总销售额: 10,642,110.80
3. 独立订单数量: 19960
4. 独立客户数量 (已识别): 4338
5. 独立产品种类 (按StockCode): 3922
6. 销售额最高的国家: United Kingdom (销售额: 9,001,744.09)
7. 数据时间范围: 从 2010-12-01 到 2011-12-09
\n请将以上摘要和生成的图表整合到您的Word报告中。
\n--- 报告生成脚本执行完毕 ---


/Users/linhengwei/.matplotlib
